In [164]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle

import data_utils
import model_utils
import train_utils
import evaluation
import submission
importlib.reload(data_utils)
importlib.reload(model_utils)
importlib.reload(train_utils)
importlib.reload(evaluation)
importlib.reload(submission)

import torch
import torch.nn as nn
from torchtext.data import TabularDataset, Field, RawField, BucketIterator, Iterator

In [165]:
DEVICE = "cuda"

### Training 

In [173]:
train_data, val_data, test_data = data_utils.prep_all_data(
    train_file="train.csv", val_file="holdout_test.csv", test_file="test.csv")

In [175]:
importlib.reload(model_utils)
m = model_utils.BaseModelNGram()
result = train_utils.train(train_data, val_data, m, device=DEVICE,
                          lr=1e-2, print_freq=1, max_epoch=5)

Epoch: 0, LR: 0.01, Train Loss: 208.0490, Val Loss: 54.5679, Val f1 0.895
Epoch: 1, LR: 0.01, Train Loss: 48.5305, Val Loss: 21.6976, Val f1 0.957
Epoch: 2, LR: 0.01, Train Loss: 20.6048, Val Loss: 8.8090, Val f1 0.989
Epoch: 3, LR: 0.01, Train Loss: 11.6165, Val Loss: 5.6757, Val f1 0.989
Epoch: 4, LR: 0.01, Train Loss: 6.7758, Val Loss: 2.3491, Val f1 0.998


In [168]:
torch.save(result["trained_model"], "./data/model_checkpoints/ngram_MLP_Jan31_all_train.mdl")

In [176]:
importlib.reload(model_utils)
m = model_utils.GRU()
result = train_utils.train(train_data, val_data, m, device=DEVICE,
                          lr=1e-2, print_freq=1, max_epoch=5)
torch.save(result["trained_model"], "./data/model_checkpoints/GRU_Jan31_all_train.mdl")

Epoch: 0, LR: 0.01, Train Loss: 298.4011, Val Loss: 152.3056, Val f1 0.623
Epoch: 1, LR: 0.01, Train Loss: 119.8285, Val Loss: 83.6388, Val f1 0.824
Epoch: 2, LR: 0.01, Train Loss: 66.6539, Val Loss: 43.1478, Val f1 0.932
Epoch: 3, LR: 0.01, Train Loss: 37.1929, Val Loss: 28.2445, Val f1 0.973
Epoch: 4, LR: 0.01, Train Loss: 21.9316, Val Loss: 17.2099, Val f1 0.984


In [177]:
train_data_phrase, val_data_phrase, test_data_phrase = data_utils.prep_all_data(
    path="./data/movie_name_condensed_data/", 
    train_file="train.csv", val_file="holdout_test.csv", test_file="test.csv")

In [178]:
m = model_utils.BaseModelNGram(path="./data/movie_name_condensed_data/")
result = train_utils.train(train_data_phrase, val_data_phrase, m, device=DEVICE,
                          lr=1e-2, print_freq=5, max_epoch=5)
torch.save(result["trained_model"], "./data/model_checkpoints/ngram_phrase_MLP_Jan31_all_train.mdl")

Epoch: 0, LR: 0.01, Train Loss: 204.9415, Val Loss: 57.7080, Val f1 0.877


In [180]:
dm = model_utils.GRU(path="./data/movie_name_condensed_data/")
result = train_utils.train(train_data_phrase, val_data_phrase, m, device=DEVICE,
                          lr=1e-2, print_freq=1, max_epoch=5)
torch.save(result["trained_model"], "./data/model_checkpoints/GRU_phrase_Jan31_all_train.mdl")

Epoch: 0, LR: 0.01, Train Loss: 17.0882, Val Loss: 5.9968, Val f1 0.989
Epoch: 1, LR: 0.01, Train Loss: 10.8891, Val Loss: 3.2211, Val f1 0.992
Epoch: 2, LR: 0.01, Train Loss: 7.0593, Val Loss: 2.8849, Val f1 0.993
Epoch: 3, LR: 0.01, Train Loss: 4.6497, Val Loss: 1.5977, Val f1 0.998
Epoch: 4, LR: 0.01, Train Loss: 4.4143, Val Loss: 2.3155, Val f1 0.995


### Ensemble

In [190]:
importlib.reload(submission)
importlib.reload(evaluation)

<module 'evaluation' from '/home/molly/Desktop/CSE244_ML_for_NLP/hw1/evaluation.py'>

In [181]:
m_gram = torch.load("./data/model_checkpoints/ngram_MLP_Jan31_all_train.mdl")
m_gram_phrase = torch.load("./data/model_checkpoints/ngram_phrase_MLP_Jan31_all_train.mdl")
m_GRU = torch.load("./data/model_checkpoints/GRU_Jan31_all_train.mdl")
m_GRU_phrase = torch.load("./data/model_checkpoints/GRU_phrase_Jan31_all_train.mdl")

In [192]:
ensemble = submission.Ensemble(models=[m_gram, m_gram_phrase, m_GRU_phrase, m_GRU], 
                               val_data=[val_data, val_data_phrase, val_data_phrase, val_data],
                               test_data=[test_data, test_data_phrase, test_data_phrase, test_data])

### Generate submission file for kaggle 

In [193]:
df = ensemble.get_ensemble_result(submission=True)
ensemble.save_submission_file("all_data")